<a href="https://colab.research.google.com/github/XuguangAi/BMI733/blob/main/AskReddit%20BERT%20V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U tensorflow-text

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
import pandas as pd

df = pd.read_csv("Reddit_balanced.csv")
df.head(5)

,Unnamed: 0,date,title,score,over_18,popularity
0,0,2021-09-23 00:52:22+00:00,"Americans that defend the confederate flag, why?",185,True,popular
1,1,2021-09-22 22:16:54+00:00,If God gave you the power to erase one thing f...,379,False,popular
2,2,2021-08-21 20:57:49+00:00,"Porn addicts,how did you end up being addicted...",119,True,popular
3,3,2021-08-21 20:53:42+00:00,"At your funeral, you can yell out one thing be...",101,False,popular
4,4,2021-08-21 18:52:58+00:00,"Without telling your age, how old are you?",240,False,popular


In [7]:
df['popularity'].value_counts()

unpopular    9999
popular      9998
Name: popularity, dtype: int64

In [8]:
df['popular']=df['popularity'].apply(lambda x: 1 if x=='popular' else 0)
df.head(5)

,Unnamed: 0,date,title,score,over_18,popularity,popular
0,0,2021-09-23 00:52:22+00:00,"Americans that defend the confederate flag, why?",185,True,popular,1
1,1,2021-09-22 22:16:54+00:00,If God gave you the power to erase one thing f...,379,False,popular,1
2,2,2021-08-21 20:57:49+00:00,"Porn addicts,how did you end up being addicted...",119,True,popular,1
3,3,2021-08-21 20:53:42+00:00,"At your funeral, you can yell out one thing be...",101,False,popular,1
4,4,2021-08-21 18:52:58+00:00,"Without telling your age, how old are you?",240,False,popular,1


In [9]:
import numpy as np

In [10]:
print('Average word length of questions in dataset is {0:.0f}.'.format(np.mean(df['title'].apply(lambda x: len(x.split())))))
print('Max word length of questions in dataset is {0:.0f}.'.format(np.max(df['title'].apply(lambda x: len(x.split())))))

Average word length of questions in dataset is 14.
Max word length of questions in dataset is 66.


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['title'],df['popular'])

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [14]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [16]:
cosine_similarity([e[0]],[e[3]])

array([[0.84703827]], dtype=float32)

In [17]:
cosine_similarity([e[3]],[e[4]])

array([[0.98720336]], dtype=float32)

In [18]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [20]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.PrecisionAtRecall(0.5, name='F1')
]


model.compile('adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [21]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
469/469 [==============================] - 112s 217ms/step - loss: 0.7067 - accuracy: 0.5188 - precision: 0.5208 - recall: 0.5366 - F1: 0.5222
Epoch 2/10
469/469 [==============================] - 101s 216ms/step - loss: 0.6879 - accuracy: 0.5512 - precision: 0.5525 - recall: 0.5649 - F1: 0.5566
Epoch 3/10
469/469 [==============================] - 100s 214ms/step - loss: 0.6792 - accuracy: 0.5688 - precision: 0.5696 - recall: 0.5825 - F1: 0.5786
Epoch 4/10
469/469 [==============================] - 100s 213ms/step - loss: 0.6724 - accuracy: 0.5773 - precision: 0.5777 - recall: 0.5922 - F1: 0.5858
Epoch 5/10
469/469 [==============================] - 100s 214ms/step - loss: 0.6720 - accuracy: 0.5791 - precision: 0.5788 - recall: 0.5984 - F1: 0.5890
Epoch 6/10
469/469 [==============================] - 100s 214ms/step - loss: 0.6638 - accuracy: 0.5965 - precision: 0.5955 - recall: 0.6159 - F1: 0.6112
Epoch 7/10
469/469 [==============================] - 100s 213ms/step - loss

In [21]:
model.evaluate(X_test, y_test)

In [22]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [23]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 0, 0, ..., 1, 1, 0])

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(y_test, y_predicted)

array([[1426, 1116],
       [ 811, 1647]])

In [25]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.64      0.56      0.60      2542
           1       0.60      0.67      0.63      2458

    accuracy                           0.61      5000
   macro avg       0.62      0.62      0.61      5000
weighted avg       0.62      0.61      0.61      5000



In [30]:
questions = [
    'You can choose one species to go extinct, what that would be?',
    'What are some tips and tricks for young adults living alone for the first time?',
]
model.predict(questions)

array([[0.36311436],
       [0.35510084]], dtype=float32)